# 04- Baseline Modeller (Logistic Regression & Random Forest)

Bu notebook’ta, **03_feature_engineering** aşamasında hazırlanmış olan `training_prepared.csv` veri seti kullanılarak ilk **baseline** modelleri kurulmaktadır.

Amaç:

- Temel bir **train / validation** şeması kurmak,
- En basit feature set (yalnızca **sayısal değişkenler**) ile
  - **Logistic Regression**
  - **Random Forest**
  modellerinin performansını görmek, 
  ileride kurulacak **XGBoost + tam feature set** modeline bir **referans** oluşturmaktır.


In [23]:
import sys
from pathlib import Path

# Şu anki çalışma klasörünü al 
CURRENT_DIR = Path().resolve()

# Eğer burada src yoksa bir üst klasöre çık
if not (CURRENT_DIR / "src").exists() and (CURRENT_DIR.parent / "src").exists():
    ROOT = CURRENT_DIR.parent
else:
    ROOT = CURRENT_DIR

# Proje kökünü sys.path'e ekle
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

print("Project root:", ROOT)
print("Src var mı?:", (ROOT / "src").exists())


Project root: C:\Users\YAĞMUR\Masaüstü\credit-risk-model
Src var mı?: True


In [24]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    f1_score,
    recall_score,
    precision_score,
    confusion_matrix,
    classification_report,
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Proje data klasörü
from src.config import DATA_DIR   # config.py içinde tanımlı

# Hazırlanmış eğitim datası (03_feature_engineering çıktısı)
TRAIN_PREPARED = DATA_DIR / "training_prepared.csv"

# Hedef kolon
TARGET_COL = "SeriousDlqin2yrs"


In [25]:
assert TRAIN_PREPARED.exists(), f"Dosya bulunamadı: {TRAIN_PREPARED}"
print("Dosya bulundu ✅:", TRAIN_PREPARED)


Dosya bulundu ✅: C:\Users\YAĞMUR\Masaüstü\credit-risk-model\data\training_prepared.csv


### 1. Eğitim Verisinin Yüklenmesi

Bu aşamada, **03_feature_engineering** çıktısı olan  
`training_prepared.csv` dosyası yüklenir.

Bu dosya:

- `02_data_cleaning` sonrasında temizlenmiş,
- `03_feature_engineering` ile feature’ları zenginleştirilmiş,
- Gereksiz / sorunlu kolonları elenmiş,

**modelleme için hazır eğitim verisini** temsil eder.  
Tüm baseline ve XGBoost deneyleri bu veri seti üzerinden yürütülecektir.



In [26]:
df = pd.read_csv(TRAIN_PREPARED)
df.shape, df.head()


((150000, 27),
    SeriousDlqin2yrs  RevolvingUtilizationOfUnsecuredLines   age  \
 0                 1                              0.766127  45.0   
 1                 0                              0.957151  40.0   
 2                 0                              0.658180  38.0   
 3                 0                              0.233810  30.0   
 4                 0                              0.907239  49.0   
 
    MonthlyIncome  NumberOfOpenCreditLinesAndLoans  \
 0         9120.0                               13   
 1         2600.0                                4   
 2         3042.0                                2   
 3         3300.0                                5   
 4        63588.0                                7   
 
    NumberRealEstateLoansOrLines  NumberOfDependents  EverDelinquent  \
 0                             6                 2.0               1   
 1                             0                 1.0               0   
 2                             0 

In [27]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 27 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   SeriousDlqin2yrs                            150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines        150000 non-null  float64
 2   age                                         150000 non-null  float64
 3   MonthlyIncome                               150000 non-null  float64
 4   NumberOfOpenCreditLinesAndLoans             150000 non-null  int64  
 5   NumberRealEstateLoansOrLines                150000 non-null  int64  
 6   NumberOfDependents                          150000 non-null  float64
 7   EverDelinquent                              150000 non-null  int64  
 8   Ever90DaysLate                              150000 non-null  int64  
 9   HighUtilizationFlag                         150000 non-null  int64  
 

### 2. Hedef Değişken Dağılımı (Class Imbalance)

Öncelikle `SeriousDlqin2yrs` hedef değişkeninin sınıf dağılımına bakıyoruz.

Bu sayede:

- Default sınıfının veri içindeki oranını,
- Veri setindeki **sınıf dengesizliği (class imbalance)** seviyesini

görmüş oluyoruz. Kredi skorlama problemlerinde default sınıfının azınlıkta olması, **metrik seçimini** doğrudan etkileyen kritik bir durumdur.



In [28]:
target_counts = df[TARGET_COL].value_counts()
target_ratio = df[TARGET_COL].value_counts(normalize=True).round(4)

target_counts, target_ratio


(SeriousDlqin2yrs
 0    139974
 1     10026
 Name: count, dtype: int64,
 SeriousDlqin2yrs
 0    0.9332
 1    0.0668
 Name: proportion, dtype: float64)

### Hedef Değişken Dağılımı Yorum

Hedef değişken **SeriousDlqin2yrs**, müşterinin önümüzdeki 2 yılda ciddi gecikme (default) yaşayıp yaşamayacağını belirtir.

Dağılım (Yaklaşık) : 

- **0 (Default değil): %93**
- **1 (Default): %7**

Veri seti belirgin şekilde **dengesizdir (imbalanced)**. Bu nedenle model değerlendirmesinde:

- **ROC-AUC**, **Recall**, **Precision**, **F1** gibi metrikler  
- Özellikle azınlık sınıfını yakalama performansı **Accuracy’den daha anlamlı olacaktır.**


### 4. Train / Validation Ayrımı (Data Leakage Kontrolü)

Model performansını doğru değerlendirmek için veri, **train (%80)** ve **validation (%20)** olarak ikiye ayrılır.

- `stratify=y` ile hedef sınıf oranı iki bölümde de korunmuştur.
- Feature engineering zaten bir önceki adımda **tüm veri** üzerinde uygulanmıştır;
  bu notebook’ta yeniden bir FE yapılmamakta, yalnızca mevcut feature’lar
  kullanılarak model kurulmaktadır.
- Modelleme sırasında fit gerektiren tüm adımlar (ör. scaler, olası encoder vb.)  **yalnızca train set üzerinde** fit edilecek, validation set üzerinde
  sadece transform uygulanacaktır.
- Bu sayede **model eğitimi tarafında ek bir data leakage riski yaratılmamaktadır.**
  FE aşamasındaki istatistiklerin tüm veri üzerinden hesaplanması ise
  teorik olarak hafif bir leakage riski taşısa da, bu durum
  `feature_engineering.md` dokümanında ayrıca tartışılmıştır.

Bu nedenle bu split işlemi, baseline değerlendirmesi için güvenilir ve yeterlidir.

In [29]:
# Hedef ve feature'ları ayır
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# Train / validation ayrımı
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

X_train.shape, X_val.shape


((120000, 26), (30000, 26))

### 5. Sayısal Değişkenlerin Seçilmesi (Baseline Aşaması)

Baseline modeller (Logistic Regression ve Random Forest) yalnızca **sayısal feature'lar** kullanılarak eğitilecektir. Bunun nedeni:

- Logistic Regression’ın kategorik değişkenleri doğrudan işleyememesi,
- scikit-learn RandomForestClassifier’ın kategorik/string değerleri otomatik dönüştürememesi,
- Encoding işlemlerinin modelleme aşamasında (tam pipeline + XGBoost) uygulanacak olmasıdır.

Bu nedenle `X_train` ve `X_val` içerisinden yalnızca sayısal kolonlar seçilmiştir.

Son durumda:
- **Train set:** 120.000 gözlem, 22 sayısal feature  
- **Validation set:** 30.000 gözlem, 22 sayısal feature  

Kategorik ve binned değişkenler, final model aşamasında uygun encoding yöntemleriyle pipeline’a dahil edilecektir.




In [30]:
# Sadece sayısal feature'ları kullanmak için kolonları seçiyoruz
numeric_cols = X_train.select_dtypes(include=[np.number]).columns

X_train_num = X_train[numeric_cols].copy()
X_val_num = X_val[numeric_cols].copy()

X_train_num.shape, X_val_num.shape, len(numeric_cols)


((120000, 22), (30000, 22), 22)

### 6. Değerlendirme Fonksiyonu

Her model için aynı metrikleri tekrar tekrar hesaplamak yerine,küçük bir yardımcı fonksiyon tanımlıyoruz.

Kullanılacak metrikler:
- ROC-AUC
- Accuracy
- Recall (default sınıfını yakalama başarısı)
- Precision
- F1-score
- Confusion matrix
- Classification report



In [31]:
def evaluate_model(name, model, X_val, y_val):
    """
    Verilen model için temel sınıflandırma metriklerini hesaplar ve ekrana basar.
    """
    y_pred = model.predict(X_val)
    y_proba = None
    try:
        y_proba = model.predict_proba(X_val)[:, 1]
    except Exception:
        # Bazı modellerde predict_proba olmayabilir
        pass

    acc = accuracy_score(y_val, y_pred)
    rec = recall_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    auc = roc_auc_score(y_val, y_proba) if y_proba is not None else np.nan

    print(f"=== {name} ===")
    print(f"Accuracy : {acc:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"F1-score : {f1:.4f}")
    print(f"ROC-AUC  : {auc:.4f}")
    print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred))
    print("\nClassification Report:\n", classification_report(y_val, y_pred))
    print("=" * 40)


### 7. Baseline Model 1 – Logistic Regression

İlk baseline modeli olarak, kredi skorlama problemlerinde yaygın şekilde kullanılan **Logistic Regression** tercih edilmiştir.

Bu aşamanın amacı **mükemmel bir sonuç üretmek değil**, daha gelişmiş modeller için **sağlam bir referans nokta (benchmark)** oluşturmaktır.

#### Neden Logistic Regression?

- Basit, hızlı ve yorumlanabilirdir.
- Model davranışını anlamak için iyi bir başlangıç noktası sunar.
- Lineer bir model olduğu için veri setinin **lineer ilişki** içerip içermediğini görmekte kullanışlıdır.
- Sınıf dengesizliğinde `class_weight="balanced"` ile azınlık sınıfa duyarlılık artırılabilir.

#### Pipeline Yapısı

- Feature’lar farklı ölçeklerde olduğu için **StandardScaler** kullanılmıştır.
- Model yalnızca **sayısal değişkenlerle** eğitilecektir.
- Kategorik değişkenler, XGBoost / final pipeline aşamasında uygun encoding ile eklenecektir.
- Logistic Regression karmaşık non-lineer ilişkileri yakalayamadığından, sonraki modeller (Random Forest, XGBoost) 
   için bir **karşılaştırma tabanı** olarak kullanılacaktır.

Bu baseline modeli, daha güçlü non-lineer modellere geçerken performans kazancını doğru yorumlayabilmemiz için bir **temel kıyaslama noktası** oluşturur.



In [32]:
log_reg_pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler(with_mean=True, with_std=True)),
        (
            "model",
            LogisticRegression(
                max_iter=1000,
                class_weight="balanced",
                n_jobs=-1,
                random_state=42,
            ),
        ),
    ]
)

# sadece sayısal kolonları kullanıyoruz
log_reg_pipeline.fit(X_train_num, y_train)

evaluate_model("LogisticRegression (baseline)", log_reg_pipeline, X_val_num, y_val)


=== LogisticRegression (baseline) ===
Accuracy : 0.8155
Recall   : 0.7456
Precision: 0.2293
F1-score : 0.3508
ROC-AUC  : 0.8622

Confusion Matrix:
 [[22971  5024]
 [  510  1495]]

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.82      0.89     27995
           1       0.23      0.75      0.35      2005

    accuracy                           0.82     30000
   macro avg       0.60      0.78      0.62     30000
weighted avg       0.93      0.82      0.86     30000



### 8. Baseline Model 2 – Random Forest

Baseline aşamasının ikinci modeli olarak, lineer modellerin yakalayamadığı **non-lineer ilişkileri öğrenebilen** ağaç tabanlı **Random Forest Classifier** kullanılmıştır.

Random Forest, Logistic Regression’a göre daha esnek bir model olup, özellikle:

- feature etkileşimlerini,
- non-lineer yapıları,
- eşik tabanlı ayrımları

daha başarılı şekilde öğrenebilir.


### Model Ayarları

Bu aşamada **hyperparameter tuning yapılmamıştır**.  
Amaç yalnızca, Random Forest’in **ham performansını** ölçmektir.

Kullanılan temel ayarlar:

- `n_estimators=200`
- `min_samples_split=5`
- `min_samples_leaf=2`
- `class_weight="balanced_subsample"`  → sınıf dengesizliğini azaltmak için


### Neden Yalnızca Sayısal Feature’lar?

Bu model de, tıpkı Logistic Regression gibi yalnızca **22 sayısal feature** ile eğitilmiştir.
Bunun başlıca nedenleri:

- scikit-learn `RandomForestClassifier` kategorik/string değişkenleri otomatik encode edemez,
- one-hot encoding yapılmadıkça kategorik değişkenleri doğrudan işleyemez,
- kategorik / binned değişkenler ilerleyen aşamalarda (özellikle XGBoost + final pipeline)
  uygun encoding ile eklenecektir.

Dolayısıyla bu aşamada Random Forest, sayısal feature seti üzerinde saf kapasitesini
ölçen bir baseline olarak kullanılmıştır.

### Bu Modelin Amacı

Bu model:

- Logistic Regression’a kıyasla **non-lineer ilişkilerden** ne kadar kazanç sağladığımızı görmek,
- Tuning yapılmadan Random Forest’ın **baseline kapasitesini** ölçmek,
- Sonraki aşamaya (XGBoost Optimization) referans oluşturmak

için değerlendirilmiştir.

Böylece, daha sonra kurulacak XGBoost modelinin gerçekten ek bir değer sağlayıp sağlamadığını adil biçimde karşılaştırabiliyoruz.


In [33]:
rf_baseline = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=2,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced_subsample",
)

# Şimdilik Logistic Regression ile aynı sayısal feature setini kullanıyoruz
rf_baseline.fit(X_train_num, y_train)

evaluate_model("RandomForest (baseline)", rf_baseline, X_val_num, y_val)


=== RandomForest (baseline) ===
Accuracy : 0.9318
Recall   : 0.3017
Precision: 0.4836
F1-score : 0.3716
ROC-AUC  : 0.8501

Confusion Matrix:
 [[27349   646]
 [ 1400   605]]

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.98      0.96     27995
           1       0.48      0.30      0.37      2005

    accuracy                           0.93     30000
   macro avg       0.72      0.64      0.67     30000
weighted avg       0.92      0.93      0.92     30000



# 9. Baseline Sonuçlarının Özeti

Baseline aşamasında iki farklı model değerlendirilmiştir:

- **Logistic Regression (scaled + class_weight="balanced")**
- **Random Forest (basit konfigürasyon)**

Bu modeller yalnızca **sayısal feature’lar** kullanılarak eğitilmiştir.  
Amaç, veri setinin **lineer mi / non-lineer mi**, **ayrıştırılabilirlik seviyesi** ve **minimum mühendislikle elde edilen performans** hakkında referans oluşturmaktır.


# Logistic Regression – Sonuç & Değerlendirme

-**Recall (1): 0.7456**
Pozitif sınıfın ~%75’i yakalanmıştır.`class_weight="balanced"` bu başarıya katkı sağlar.

-**Precision (1): 0.229**
Yanlış pozitifler çoktur → model fazla alarm üretir. Imbalanced veri için lineer modellerde sık görülen bir davranıştır.

-**F1-score (1): 0.3508**
Recall yüksek olsa da precision düşük olduğu için F1 sınırlıdır.

-**ROC-AUC: 0.8622**
Olasılık bazında ayrıştırma gücü oldukça iyidir.

### **Confusion Matrix – Özet**
- **TP = 1495**
- **FP = 5024**
- **FN = 510**
- **TN = 22,971**

**Genel olarak:**
- Pozitif sınıfı yakalama kapasitesi güçlü (yüksek recall)
- Ancak yanlış alarm oranı yüksek (düşük precision)
- Lineer yapı veri setinin karmaşıklığını tam yakalayamıyor


# Random Forest – Sonuç & Değerlendirme

-***Recall (1): 0.3017**
Pozitif sınıfın sadece %30’u yakalanmıştır.Numeric-only feature set, RF’nin potansiyelini sınırlar.

-***Precision (1): 0.4836**
LR’a göre belirgin şekilde daha yüksektir. Daha az yanlış alarm üretir.

-**F1-score (1): 0.3716**
Recall düşük olsa da precision yüksek olduğu için LR’dan daha iyidir.

-**ROC-AUC: 0.8501**
Ağaç tabanlı model için iyi bir ayrıştırma gücüdür.

### **Confusion Matrix – Özet**
- **TP = 605**
- **FP = 646**
- **FN = 1,400**
- **TN = 27,349**

**Genel olarak:**
- Pozitif sınıfı yakalama başarısı düşük (low recall)
- Tahmin kalitesi daha yüksek (high precision)
- Destekleyici ama eksik bir baseline performansı


# Genel Baseline Değerlendirmesi

1. **Veri tam olarak lineer değildir.**  
   LR yüksek recall sağlarken precision’da ciddi düşüş yaşıyor; RF ise non-lineer yapıları kullanarak farklı bir precision/recall dengesi sunuyor.

2. **Sayısal feature setinin tahmin gücü sınırlıdır.**  
   Non-lineer RF modeli bile recall’da düşük kalmaktadır.

3. **Class imbalance etkisi büyüktür.**  
   Accuracy **tek başına** anlamlı bir metrik değildir.

4. **ROC-AUC her iki modelde de ~0.85 seviyesinde.**  
   Veri orta–güçlü ayrıştırılabilirliğe sahip.

5. **Gerçek performans için kategorik ve binned feature’lar şarttır.**  
   Encoding yapılmadığı için modeller potansiyellerine ulaşamamıştır.


# Bu Baseline Ne Gösteriyor? 

- Logistic Regression → **yüksek recall**, **çok düşük precision**
- Random Forest → **düşük recall**, **yüksek precision**
- Her iki model de sınırlı bilgiyle çalışıyor
- Gerçek performans, tam feature engineering + encoding + boosting modellerinde ortaya çıkacak


# Sonraki Adım

- Tüm feature engineering çıktılarının modele dahil edilmesi  
- Encoding stratejisinin kurulması  
- XGBoost optimizasyonu  
- Threshold tuning  
- SHAP analizi  

**Baseline → referans noktasıdır.  
Final model → gerçek mühendislik performansıdır.**
